#Part 1A: Data Cleaning#
Our goal in the data processing component of the project was to format information from the 5 independent variables and 2 dependent variables (listed in the initial questions section). Each of the data for these different categories was pulled from a different data source (url’s for each source are in the appendix), so we had to read in and clean 7 different CSV files (all in the data folder within our repository). Since our desired output was one CSV file with each row containing information for all 7 variable categories for a given school, we needed to standardize each data set such that one column contained a school’s District Bureau Number (DBN) since it represented a unique ID for each school, and other columns with appropriate data for that school. Because we were pulling from a variety of data sets, however, we needed to perform various cleaning steps and modifications on each of the following data sets so that they we could merge them properly into one uniform data set.

In [5]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import math
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [6]:
#import all data sets
Math=pd.read_csv("data/Math.csv")
Charter = pd.read_csv("data/Math-Charter.csv")
Size = pd.read_csv("data/ClassSize.csv")
Survey = pd.read_csv("data/SurveyData.csv")
Demo = pd.read_csv("data/DemographicData.csv")
funding = pd.read_csv("data/fundingdf.csv")
Safety = pd.read_csv("data/School_Safety_Report.csv")

###Math Scores, Public and Charter Data
The data we used for math test scores came from two files, one with only public school scores and one with charter school scores. For both files, we first eliminated the irrelevant demographic columns and only kept data from the year 2011. Then we used the fact that the scores were already segregated based on school type to determine each school’s type. To do this, we created a column called “School Type”, and we set the value for all of the rows to “Charter” in the charter dataframe and “Public” in the public dataframe. Since these dataframes had the same columns (different metrics for math test performance for each grade in each DBN identified school), we appended the two together to form one large dataframe (merge1).

In [8]:
#CLEAN UP MATH DATA
#drop irrelevant demographic info and years
Mathdf = Math.drop("Demographic", axis = 1)
Mathdf =  Mathdf[Mathdf["Year"] == 2011]
#set school type to public because schools in the math.csv were only from public schools
Mathdf["School Type"] = "Public"
Mathdf.head()

,DBN,Grade,Year,Number Tested,Mean Scale Score,Num Level 1,Pct Level 1,Num Level 2,Pct Level 2,Num Level 3,Pct Level 3,Num Level 4,Pct Level 4,Num Level 3 and 4,Pct Level 3 and 4,School Type
5,01M015,3,2011,28,671,10,35.7,13,46.4,5,17.9,0,0,5,17.9,Public
12,01M015,4,2011,28,668,3,10.7,14,50,9,32.1,2,7.1,11,39.3,Public
19,01M015,5,2011,25,667,5,20,8,32,12,48,0,0,12,48,Public
27,01M015,All Grades,2011,81,669,18,22.2,35,43.2,26,32.1,2,2.5,28,34.6,Public
34,01M019,3,2011,34,679,2,5.9,23,67.6,7,20.6,2,5.9,9,26.5,Public


In [7]:
#CLEAN UP CHARTER DATA
#drop irrelevant demographic info and years
Charter = Charter[Charter["Year"] == 2011]
Charterdf = Charter.drop("Demographic", axis = 1)
#set school type to charter because schools in charter.csv were only from charter schools
Charterdf["School Type"] = "Charter"
Charterdf.head()

,DBN,Grade,Year,Number Tested,Mean Scale Score,Num Level 1,Pct Level 1,Num Level 2,Pct Level 2,Num Level 3,Pct Level 3,Num Level 4,Pct Level 4,Num Level 3 and 4,Pct Level 3 and 4,School Type
5,84K355,5,2011,79,696,1,1.3,12,15.2,39,49.4,27,34.2,66,83.5,Charter
11,84K355,6,2011,71,714,0,0,2,2.8,14,19.7,55,77.5,69,97.2,Charter
16,84K355,7,2011,68,706,0,0,0,0,14,20.6,54,79.4,68,100,Charter
20,84K355,8,2011,55,718,0,0,0,0,10,18.2,45,81.8,55,100,Charter
27,84K355,All Grades,2011,273,708,1,0.4,14,5.1,77,28.2,181,66.3,258,94.5,Charter


In [9]:
#merge math data and charter data
merge1 = Mathdf.append(Charterdf, ignore_index=True)
merge1.rename(columns={'Mean Scale Score':'Mean_Scale_Score'}, inplace=True)
# convert meanscalescores into ints
counter = 0
for i in merge1.iterrows():
    if i[1]["Mean_Scale_Score"] == "s":
        merge1.ix[counter, "Mean_Scale_Score"] = "NaN"
    else:
        merge1.ix[counter, "Mean_Scale_Score"]= int(i[1]["Mean_Scale_Score"])
    counter += 1
type(merge1.iloc[0]["Mean_Scale_Score"])
merge1.head()

,DBN,Grade,Year,Number Tested,Mean_Scale_Score,Num Level 1,Pct Level 1,Num Level 2,Pct Level 2,Num Level 3,Pct Level 3,Num Level 4,Pct Level 4,Num Level 3 and 4,Pct Level 3 and 4,School Type
0,01M015,3,2011,28,671,10,35.7,13,46.4,5,17.9,0,0,5,17.9,Public
1,01M015,4,2011,28,668,3,10.7,14,50,9,32.1,2,7.1,11,39.3,Public
2,01M015,5,2011,25,667,5,20,8,32,12,48,0,0,12,48,Public
3,01M015,All Grades,2011,81,669,18,22.2,35,43.2,26,32.1,2,2.5,28,34.6,Public
4,01M019,3,2011,34,679,2,5.9,23,67.6,7,20.6,2,5.9,9,26.5,Public


###Class Size Data
This data did not identify each school by DBN, but instead had a column with the city’s district (CSD) and a school code. Since DBN’s are just a combined string of a 0, the CSD, then the school code, we had to create a new column in this dataframe that concatenated the appropriate values to form the DBN for each school. Additionally, we planned to merge our dataframes on both DBN and grade (so distinction between data about different grades would be preserved), we needed to standardize the grade values in this dataframe to match those of merge1. The grades in this dataframe had 0’s before the main value (e.g. “03” instead of “3”) so we removed those. Then to ensure that we could run our regression on the class size values, we converted the strings to floats. Next we dropped columns we did not plan to use in our regression, and only kept data regarding the “GEN ED” program since it was the most generic class type and thus most representative sample. Finally, we altered the columns names (i.e. removed capitalization) to match those of merge1. This allowed us to merge the class size data set with merge1 based on matching DBN’s and Grades, forming a merge2 dataframe.


In [10]:
#CLEAN UP SIZE DATA
#Standardize DBN numbers for class size data
Size["DBN"] = "N/A"
counter = 0
for i in Size.iterrows():
    #concatenate school district and code to form dbn numbers
    newdbn = "0"+str(i[1]["CSD"])+i[1]["SCHOOL CODE"]
    Size.ix[counter, "DBN"]= newdbn
    #take only sencond number of grades with a 0 added to the beginning
    if type(i[1]["GRADE "]) == float:
        grade = "N/A"
    elif len(i[1]["GRADE "]) == 2:
        grade = i[1]["GRADE "][1:]
    Size.ix[counter, "GRADE "] = grade
    if math.isnan((i[1]["AVERAGE CLASS SIZE"])):
        continue
    else:
        Size.ix[counter, "AVERAGE CLASS SIZE"] = float(i[1]["AVERAGE CLASS SIZE"])
    counter += 1

#Only keep relevant class size columns
sizeallcolumns = Size.columns 
sizedropcolumns = []
# print len(sizeallcolumns)
for title in sizeallcolumns:
    if title in ["SCHOOL NAME", "BOROUGH", "GRADE ", 'PROGRAM TYPE', "AVERAGE CLASS SIZE", "DBN"]:
        continue
    else:
        sizedropcolumns.append(title)

#Keep only gen ed samples to standardize classroom type we gather data from
Sizedf = Size.drop(sizedropcolumns, axis = 1)
Sizedf =  Sizedf[Sizedf["PROGRAM TYPE"] == "GEN ED"]

#Modify format of column names so they match others while merging
Sizedf.columns = ['School Name', 'Borough', 'Grade', 'Program Type', 'Average Class Size', 'DBN']
Sizedf.head()

,School Name,Borough,Grade,Program Type,Average Class Size,DBN
0,M,P.S. 015 Roberto Clemente,K,GEN ED,19,01M015
2,M,P.S. 015 Roberto Clemente,1,GEN ED,17,01M015
4,M,P.S. 015 Roberto Clemente,2,GEN ED,15,01M015
6,M,P.S. 015 Roberto Clemente,3,GEN ED,12,01M015
8,M,P.S. 015 Roberto Clemente,4,GEN ED,13,01M015


In [11]:
#merge class size data into other merged data
merge2 = merge1.merge(Sizedf, on = ["DBN", "Grade"], how = "left")

###Demographic Data
After eliminating unnecessary columns and data from the wrong timeframe, we converted the demographic information into a format that could be processed in the regression. Because the numbers were strings rather than floats, we needed to remove the percentage symbol so we could then convert it into a float. Also, the values were percentages but were represented out of 100 rather than out of 1, so we then divided the floats by 100. Since this dataframe had correct DBN values and was not segregated by grade, we could then merge it with merge2 to form a merge3 dataframe.


In [12]:
#Clean up years for demographic data
Demog =  Demo[Demo["Year"] == "2010-11"]

#Only keep relevant demographic columns
demogallcolumns = Demog.columns 
demogdropcolumns = []
for title in demogallcolumns:
    if title in ["DBN", "School Name", "% Female", "% Male", "% Asian", "% Black", "% Hispanic", "% Other", "% White"]:
        continue
    else:
        demogdropcolumns.append(title)

#drop irrelvant columns
Demogdf = Demog.drop(demogdropcolumns, axis = 1)

#convert percentages into floats without percent sign
for i in xrange(0, len(Demogdf["DBN"])):
    Demogdf.iloc[i]["% Female"] = float(Demogdf.iloc[i]["% Female"][:-1])/100
    Demogdf.iloc[i]["% Male"] = float(Demogdf.iloc[i]["% Male"][:-1])/100
    Demogdf.iloc[i]["% Asian"] = float(Demogdf.iloc[i]["% Asian"][:-1])/100
    Demogdf.iloc[i]["% Black"] = float(Demogdf.iloc[i]["% Black"][:-1])/100
    Demogdf.iloc[i]["% Hispanic"] = float(Demogdf.iloc[i]["% Hispanic"][:-1])/100
    Demogdf.iloc[i]["% Other"] = float(Demogdf.iloc[i]["% Other"][:-1])/100
    Demogdf.iloc[i]["% White"] = float(Demogdf.iloc[i]["% White"][:-1])/100


In [13]:
#merge demographic data with other merged data
merge3 = merge2.merge(Demogdf, on = "DBN", how = "left")
merge3.head()

,DBN,Grade,Year,Number Tested,Mean_Scale_Score,Num Level 1,Pct Level 1,Num Level 2,Pct Level 2,Num Level 3,Pct Level 3,Num Level 4,Pct Level 4,Num Level 3 and 4,Pct Level 3 and 4,School Type,School Name_x,Borough,Program Type,Average Class Size,School Name_y,% Female,% Male,% Asian,% Black,% Hispanic,% Other,% White
0,01M015,3,2011,28,671,10,35.7,13,46.4,5,17.9,0,0,5,17.9,Public,M,P.S. 015 Roberto Clemente,GEN ED,12.0,P.S. 015 Roberto Clemente,0.443,0.557,0.064,0.369,0.542,0.005,0.02
1,01M015,4,2011,28,668,3,10.7,14,50,9,32.1,2,7.1,11,39.3,Public,M,P.S. 015 Roberto Clemente,GEN ED,13.0,P.S. 015 Roberto Clemente,0.443,0.557,0.064,0.369,0.542,0.005,0.02
2,01M015,5,2011,25,667,5,20,8,32,12,48,0,0,12,48,Public,M,P.S. 015 Roberto Clemente,GEN ED,27.0,P.S. 015 Roberto Clemente,0.443,0.557,0.064,0.369,0.542,0.005,0.02
3,01M015,All Grades,2011,81,669,18,22.2,35,43.2,26,32.1,2,2.5,28,34.6,Public,NaN,NaN,NaN,NaN,P.S. 015 Roberto Clemente,0.443,0.557,0.064,0.369,0.542,0.005,0.02
4,01M019,3,2011,34,679,2,5.9,23,67.6,7,20.6,2,5.9,9,26.5,Public,M,P.S. 019 Asher Levy,GEN ED,17.5,P.S. 019 Asher Levy,0.527,0.473,0.146,0.271,0.473,0.015,0.095


###Safety and survey data
Both of these dataframes already had proper DBN numbers and formatting. After modifying one column header (dbn) to capitalize the letters such that it could be merged with the DBN column in merge3, we dropped columns and rows with extraneous data and merged them with merge3, ultimately creating a merge5 dataframe.

In [14]:
#clean safety data
#keep only relevant columns
safetyallcolumns = Safety.columns 
safetydropcolumns = []
for title in safetyallcolumns:
    if title in ["DBN", "AvgOfMajor N", "AvgOfVio N", "AvgOfNoCrim N"]:
        continue
    else:
        safetydropcolumns.append(title)
Safetydf = Safety.drop(safetydropcolumns, axis = 1)
Safetydf.head()

,DBN,AvgOfMajor N,AvgOfNoCrim N,AvgOfVio N
0,15K001,0.86,5.55,1.29
1,17K002,NaN,NaN,NaN
2,75K141,NaN,NaN,NaN
3,84K704,NaN,NaN,NaN
4,NaN,0.52,2.49,0.75


In [15]:
#merge safety data with other data
merge4 = merge3.merge(Safetydf, on = "DBN", how = "left")
merge4.head()

,DBN,Grade,Year,Number Tested,Mean_Scale_Score,Num Level 1,Pct Level 1,Num Level 2,Pct Level 2,Num Level 3,Pct Level 3,Num Level 4,Pct Level 4,Num Level 3 and 4,Pct Level 3 and 4,School Type,School Name_x,Borough,Program Type,Average Class Size,School Name_y,% Female,% Male,% Asian,% Black,% Hispanic,% Other,% White,AvgOfMajor N,AvgOfNoCrim N,AvgOfVio N
0,01M015,3,2011,28,671,10,35.7,13,46.4,5,17.9,0,0,5,17.9,Public,M,P.S. 015 Roberto Clemente,GEN ED,12.0,P.S. 015 Roberto Clemente,0.443,0.557,0.064,0.369,0.542,0.005,0.02,0.43,1.23,0.41
1,01M015,4,2011,28,668,3,10.7,14,50,9,32.1,2,7.1,11,39.3,Public,M,P.S. 015 Roberto Clemente,GEN ED,13.0,P.S. 015 Roberto Clemente,0.443,0.557,0.064,0.369,0.542,0.005,0.02,0.43,1.23,0.41
2,01M015,5,2011,25,667,5,20,8,32,12,48,0,0,12,48,Public,M,P.S. 015 Roberto Clemente,GEN ED,27.0,P.S. 015 Roberto Clemente,0.443,0.557,0.064,0.369,0.542,0.005,0.02,0.43,1.23,0.41
3,01M015,All Grades,2011,81,669,18,22.2,35,43.2,26,32.1,2,2.5,28,34.6,Public,NaN,NaN,NaN,NaN,P.S. 015 Roberto Clemente,0.443,0.557,0.064,0.369,0.542,0.005,0.02,0.43,1.23,0.41
4,01M019,3,2011,34,679,2,5.9,23,67.6,7,20.6,2,5.9,9,26.5,Public,M,P.S. 019 Asher Levy,GEN ED,17.5,P.S. 019 Asher Levy,0.527,0.473,0.146,0.271,0.473,0.015,0.095,NaN,NaN,NaN


In [16]:
#CLEAN UP SURVEY DATA
# SurveyWhole = Survey.append(Survey75, ignore_index=True)
allcolumns = Survey.columns 
dropcolumns = []
for title in allcolumns:
    #create array of columns to drop in dropcolumns array
    if title in ["dbn", "schoolname", "eng_p_11", "eng_s_11", "eng_t_11", "aca_p_11", "aca_t_11", "aca_s_11"]:
        continue   
    else:
        dropcolumns.append(title)
#drop irrelevant columns and edit column format so they can merge
Surveydf = Survey.drop(dropcolumns, axis = 1)
Surveydf.rename(columns={'dbn':'DBN'}, inplace=True)
Surveydf.head()


,DBN,schoolname,eng_p_11,aca_p_11,eng_t_11,aca_t_11,eng_s_11,aca_s_11
0,01M015,P.S. 015 Roberto Clemente,7.5,7.8,7.6,7.9,NaN,NaN
1,01M019,P.S. 019 Asher Levy,7.6,7.8,8.9,9.1,NaN,NaN
2,01M020,P.S. 020 Anna Silver,8.3,8.6,6.8,7.5,NaN,NaN
3,01M034,P.S. 034 Franklin D. Roosevelt,8.0,8.5,6.8,7.8,6.5,7.4
4,01M063,P.S. 063 William McKinley,8.1,7.9,7.8,8.1,NaN,NaN


In [17]:
merge5 = merge4.merge(Surveydf, on = "DBN", how = "left")
merge5.head()

,DBN,Grade,Year,Number Tested,Mean_Scale_Score,Num Level 1,Pct Level 1,Num Level 2,Pct Level 2,Num Level 3,Pct Level 3,Num Level 4,Pct Level 4,Num Level 3 and 4,Pct Level 3 and 4,School Type,School Name_x,Borough,Program Type,Average Class Size,School Name_y,% Female,% Male,% Asian,% Black,% Hispanic,% Other,% White,AvgOfMajor N,AvgOfNoCrim N,AvgOfVio N,schoolname,eng_p_11,aca_p_11,eng_t_11,aca_t_11,eng_s_11,aca_s_11
0,01M015,3,2011,28,671,10,35.7,13,46.4,5,17.9,0,0,5,17.9,Public,M,P.S. 015 Roberto Clemente,GEN ED,12.0,P.S. 015 Roberto Clemente,0.443,0.557,0.064,0.369,0.542,0.005,0.02,0.43,1.23,0.41,P.S. 015 Roberto Clemente,7.5,7.8,7.6,7.9,NaN,NaN
1,01M015,4,2011,28,668,3,10.7,14,50,9,32.1,2,7.1,11,39.3,Public,M,P.S. 015 Roberto Clemente,GEN ED,13.0,P.S. 015 Roberto Clemente,0.443,0.557,0.064,0.369,0.542,0.005,0.02,0.43,1.23,0.41,P.S. 015 Roberto Clemente,7.5,7.8,7.6,7.9,NaN,NaN
2,01M015,5,2011,25,667,5,20,8,32,12,48,0,0,12,48,Public,M,P.S. 015 Roberto Clemente,GEN ED,27.0,P.S. 015 Roberto Clemente,0.443,0.557,0.064,0.369,0.542,0.005,0.02,0.43,1.23,0.41,P.S. 015 Roberto Clemente,7.5,7.8,7.6,7.9,NaN,NaN
3,01M015,All Grades,2011,81,669,18,22.2,35,43.2,26,32.1,2,2.5,28,34.6,Public,NaN,NaN,NaN,NaN,P.S. 015 Roberto Clemente,0.443,0.557,0.064,0.369,0.542,0.005,0.02,0.43,1.23,0.41,P.S. 015 Roberto Clemente,7.5,7.8,7.6,7.9,NaN,NaN
4,01M019,3,2011,34,679,2,5.9,23,67.6,7,20.6,2,5.9,9,26.5,Public,M,P.S. 019 Asher Levy,GEN ED,17.5,P.S. 019 Asher Levy,0.527,0.473,0.146,0.271,0.473,0.015,0.095,NaN,NaN,NaN,P.S. 019 Asher Levy,7.6,7.8,8.9,9.1,NaN,NaN


###Funding Data
After scraping our funding data (see GetFunding.ipynb for code and explanation of Part 1B of our process, which describes scraping) we returned to MergeFiles.ipynb and read in fundingdf.csv into our file. Since it had DBN numbers that matched those in merge5, we could merge it with merge5 to form a final mergedf. 

In [18]:
#Clean up funding columns
funding.columns = ["DBN", "Funding"]
funding.head()

,DBN,Funding
0,10X225,3001810
1,14K318,7720906
2,10X226,2722452
3,01M110,2559804
4,05M197,2270648


In [20]:
mergedf = merge5.merge(funding, on = "DBN", how = "left")

###Final Cleaning and Exportation
Next, we returned to MergeFiles.ipynb and read in fundingdf.csv into our file. Since it had DBN numbers that matched those in merge5, we could merge it with merge5 to form a final mergedf. When we exported this file and ran our regressions on it, however, we realized that some of the rows of data had NaN values. Hence we went back to our MergeFiles code and added code that would iterate through the rows of all of the relevant columns and replace NaN values with the average values for that corresponding column. Moreover, the regression was thrown off by funding values of 0 (which had been set as the default value if scraping did not work), so we eliminated those rows since they provided inaccurate funding information. Finally, we realized while analyzing test scores for each grade that data for “All Grades” was just a mean value for all the grades for a given school. Hence this data was redundant (since it provided an average value for data already present in our dataset), and therefore we eliminated those rows. After making these modifications, we then exported the final cleaned dataframe as mergedf_final.csv, which we then used for our regression.

In [22]:
#REPLACE NAN VALUES IN EACH COLUMN WITH OVERALL MEAN FOR THAT COLUMN
#replace NAN values in Mean_Scale_Score column
array = []
#create array with all values that are not NAN
for i in mergedf.iterrows():
    if i[1]["Mean_Scale_Score"] != "NaN":
        array.append(i[1]["Mean_Scale_Score"])

#store value for averall mean of Mean_Scale_Score
mean_avg = np.mean(array)

#iterate through mergedf columns, replacing NaN values with overall mean value
counter = 0
for i in mergedf.iterrows():
    if i[1]["Mean_Scale_Score"] == "NaN":
        #replace NaN value with average
        mergedf.ix[counter, "Mean_Scale_Score"] = mean_avg
    counter +=1

#repeat same process for class size column
array_size = []
for i in mergedf.iterrows():
    if ~np.isnan(i[1]["Average Class Size"]):
        array_size.append(i[1]["Average Class Size"])
size_average = np.mean(array_size)
counter1 = 0
for i in mergedf.iterrows():
    if np.isnan(i[1]["Average Class Size"]):
        mergedf.ix[counter1, "Average Class Size"] = size_average
    counter1 +=1

#repeat same process for all demographic data
array_f = []
for i in mergedf.iterrows():
    if ~np.isnan(i[1]["% Female"]):
        array_f.append(i[1]["% Female"])
female_avg = np.mean(array_f)
counter2 = 0
for i in mergedf.iterrows():
    if np.isnan(i[1]["% Female"]):
        mergedf.ix[counter2, "% Female"] = female_avg
    counter2 +=1

array_asian = []
counter3 = 0
for i in mergedf.iterrows():
    if ~np.isnan(i[1]["% Asian"]):
        array_asian.append(i[1]["% Asian"])
asian_avg = np.mean(array_asian)
for i in mergedf.iterrows():
    if np.isnan(i[1]["% Asian"]):
        mergedf.ix[counter3, "% Asian"] = asian_avg
    counter3 +=1

array_black = []
counter4 = 0
for i in mergedf.iterrows():
    if ~np.isnan(i[1]["% Black"]):
        array_black.append(i[1]["% Black"])
black_avg = np.mean(array_black)
for i in mergedf.iterrows():
    if np.isnan(i[1]["% Black"]):
        mergedf.ix[counter4, "% Black"] = black_avg
    counter4 +=1

array_his = []
counter5 = 0
for i in mergedf.iterrows():
    if ~np.isnan(i[1]["% Hispanic"]):
        array_his.append(i[1]["% Hispanic"])
his_avg = np.mean(array_his)
for i in mergedf.iterrows():
    if np.isnan(i[1]["% Hispanic"]):
        mergedf.ix[counter5, "% Hispanic"] = his_avg
    counter5 +=1
    
array_crime = []
counter6 = 0
for i in mergedf.iterrows():
    if ~np.isnan(i[1]["AvgOfNoCrim N"]):
        array_crime.append(i[1]["AvgOfNoCrim N"])
crime_avg = np.mean(array_crime)
for i in mergedf.iterrows():
    if np.isnan(i[1]["AvgOfNoCrim N"]):
        mergedf.ix[counter6, "AvgOfNoCrim N"] = crime_avg
    counter6 +=1
    
array_other=[]
for i in mergedf.iterrows():
    if ~np.isnan(i[1]["% Other"]):
        array_other.append(i[1]["% Other"])
counter7 =0
other_avg = np.mean(array_other)
for i in mergedf.iterrows():
    if np.isnan(i[1]["% Other"]):
        mergedf.ix[counter7, "% Other"] = other_avg
    counter7 +=1
    
array_survey=[]
for i in mergedf.iterrows():
    if ~np.isnan(i[1]["aca_s_11"]):
        array_survey.append(i[1]["aca_s_11"])
counter8= 0
survey_avg= np.mean(array_survey)
for i in mergedf.iterrows():
    if np.isnan(i[1]["aca_s_11"]):
        mergedf.ix[counter8, "aca_s_11"] = survey_avg
    counter8 +=1
    
array_white = []
for i in mergedf.iterrows():
    if ~np.isnan(i[1]["% White"]):
        array_white.append(i[1]["% White"])
counterw = 0
white_avg = np.mean(array_white)
for i in mergedf.iterrows():
    if np.isnan(i[1]["% White"]):
        mergedf.ix[counterw, "% White"] = white_avg
    counterw +=1

In [28]:
#standardize column names so they work with the regression model (i.e. no spaces)
mergedf.columns = ['DBN', 'Grade', 'Year', 'Number_Tested', 'Mean_Scale_Score', 'Num_Level1', 'Pct_Level1', 'Num_Level2', 'Pct_Level2', 'Num_Level3', 'Pct_Level3', 'Num_Level4', 'Pct_Level4', 'Num_Level3_and4', 'Pct_Level3_and4', 'School_Type', 'School_Name_x', 'Borough', 'Program_Type', 'Average_Class_Size', 'School_Name_y', 'Female_Percentage', 'Male_Percentage', 'Asian_Percentage', 'Black_Percentage', 'Hispanic_Percentage', 'Other_Percentage', 'White_Percentage', 'Avg_Major_N', 'Avg_No_Crim_N', 'Avg_Vio_N', 'School_Name', 'Eng_p_11', 'Aca_p_11', 'Eng_t_11', 'Aca_t_11', 'Eng_s_11', 'Aca_s_11', 'Funding']

In [29]:
#eliminate rows where funding is 0, or that only have average class data rather than data for a grade
mergedf =  mergedf[mergedf["Funding"] != 0]
newdf = mergedf[mergedf["Grade"] != "All Grades"]

In [30]:
#export file to a csv
newdf.to_csv("mergedf_final.csv", index=False)